In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

In [ ]:
conf = (
    SparkConf()
    .setAppName("TEEHR")
    .setMaster("local[*]")
    # .set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")
    # .set("spark.sql.execution.arrow.pyspark.enabled", "true")
    # .set("spark.sql.session.timeZone", "UTC")
    # .set("spark.driver.host", "localhost")
    .set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.4.1")
    # .set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1")
    # .set("spark.jars.packages", "com.amazonaws:aws-java-sdk-bundle:1.12.787")
    # .set("spark.sql.parquet.enableVectorizedReader", "false")
)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
options = {'header': 'true', 'ignoreMissingFiles': 'true'}
path = 's3a://ciroh-rti-public-data/teehr-data-warehouse/v0_4_evaluations/e0_2_location_example/dataset/units/units.csv'

In [ ]:
spark.read.format("csv").options(**options).load(path)

In [ ]:
spark.read.format("csv").load(path)

In [ ]:
path